# Recon2.2plusthreeCompetitionII.json [Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba
from utils import findBiomarkers

In [2]:
M1 = cobra.io.load_json_model('Recon3DplusthreeCompetitive(previous).json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [21]:
M=M1.copy()

In [22]:
# deleting brain medium composition because this metabolite will uptake from blood
M.reactions.EX_co2_e_INB.lower_bound=0
M.reactions.EX_glc_D_e_INB.lower_bound=0
M.reactions.EX_h_e_INB.lower_bound=0
M.reactions.EX_h2o_e_INB.lower_bound=0
M.reactions.EX_hco3_e_INB.lower_bound=0
M.reactions.EX_his_L_e_INB.lower_bound=0
M.reactions.EX_ile_L_e_INB.lower_bound=0
M.reactions.EX_leu_L_e_INB.lower_bound=0
M.reactions.EX_lys_L_e_INB.lower_bound=0
M.reactions.EX_met_L_e_INB.lower_bound=0
M.reactions.EX_nh4_e_INB.lower_bound=0
M.reactions.EX_o2_e_INB.lower_bound=0
M.reactions.EX_phe_L_e_INB.lower_bound=0
M.reactions.EX_pi_e_INB.lower_bound=0
M.reactions.EX_so4_e_INB.lower_bound=0
M.reactions.EX_thr_L_e_INB.lower_bound=0
M.reactions.EX_trp_L_e_INB.lower_bound=0
M.reactions.EX_val_L_e_INB.lower_bound=0
M.reactions.EX_tyr_L_e_INB.lower_bound=0
M.reactions.EX_hco3_e.lower_bound=0
# deleting liver and brain secreted reactions which no metabolites could not secrete in liver and brain.
SecretedRXN=[rxn for rxn in M.reactions if M.reactions.get_by_id(rxn.id).upper_bound>0 and 'EX_' in rxn.id]
for reaction in SecretedRXN:
    M.reactions.get_by_id(reaction.id).upper_bound=0

In [23]:
M.reactions.EX_TH_B_INB.upper_bound=0

In [24]:
BloodMeta1=[meta for meta in M.metabolites if '_BD' in meta.id
            and '34dhphe' not in meta.id 
            and 'phpyr_BD'not in meta.id]
len(BloodMeta1)

32

In [25]:
## phenylpyruvate and levodopa (34dhphe_BD) alreadly exist
BloodMetaExist=['glc_D_c','phe_L_c','tyr_L_c','his_L_c','ile_L_c','leu_L_c','met_L_c',
          'thr_L_c','trp_L_c','val_L_c','lys_L_c','ala_L_c','arg_L_c','asn_L_c','asp_L_c','cys_L_c',
          'glu_L_c','gln_L_c','gly_c','pro_L_c','ser_L_c','o2_c','nh4_c','pi_c','34dhphe_c','phpyr_c']

In [26]:
MetaCystol=[metab for metab in M.metabolites if metab.compartment == 'c'and '_INB' not in metab.id]
len(MetaCystol)

2125

In [27]:
SPECIFIC=['11_cis_retfa','retinol_cis_11','9_cis_retfa','retinol_9_cis','13_cis_oretn','13_cis_retn',
          'retinal_cis_13','retinal_11_cis','retinal_cis_9','retinol_cis_13','13_cis_retnglc']

In [28]:
for META in SPECIFIC:
    META_c=META + '_c'
    META_c_INB=META + '_c'+'_INB'
    from cobra import Model, Reaction, Metabolite
    # Adding reaction from liver to blood
    rxn_id = f'{META}_TOBD'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {META} from liver to blood'
    reaction.lower_bound = -1000  
    reaction.upper_bound = 1000

    # Create metabolite object
    meta = Metabolite(f'{META}_BD', formula='', name='', compartment='blood')
    meta2=M.metabolites.get_by_id(META_c)
    reaction.add_metabolites({meta: 1.0, meta2:-1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])
    
    # Adding reaction from blood to brain
    rxn_id = f'{META}_TOB'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {META} from blood to brain'
    reaction.lower_bound = -1000  
    reaction.upper_bound = 1000

    # Create metabolite object
    meta = Metabolite(f'{META}_BD', formula='', name='', compartment='blood')
    meta3=M.metabolites.get_by_id(META_c_INB)
    reaction.add_metabolites({meta: -1.0, meta3:1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])

In [29]:
## Phenylpyruvate and leveedopa are in MeetaCystol.
MetaCystol=[metab for metab in M.metabolites if metab.compartment == 'c' and '_cis' not in metab.id
           and '_car' not in metab.id and '_INB' not in metab.id and metab.id not in BloodMetaExist]
len(MetaCystol)

2088

In [30]:
for META in MetaCystol:
    meta_id_c = META.id.replace("_c","")
    meta_id_c1 = META.id.replace("_c","_c_INB")
    from cobra import Model, Reaction, Metabolite
    # Adding reaction from liver to blood
    rxn_id = f'{meta_id_c}_TOBD'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {meta_id_c} from liver to blood'
    ## Irrevisble because they are too many oligopeptide in the model which will make amino acid cross the BBB
    reaction.lower_bound = 0  
    reaction.upper_bound = 1000

    # Create metabolite object
    meta = Metabolite(f'{meta_id_c}_BD', formula='', name='', compartment='blood')
    meta2=M.metabolites.get_by_id(META.id)
    reaction.add_metabolites({meta: 1.0, meta2:-1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])
    
    # Adding reaction from blood to brain
    rxn_id = f'{meta_id_c}_TOB'
    reaction = Reaction(rxn_id)
    reaction.name = f'Transport reaction of {meta_id_c} from blood to brain'
    reaction.lower_bound = -1000
    reaction.upper_bound = 0

    # Create metabolite object
    meta = Metabolite(f'{meta_id_c}_BD', formula='', name='', compartment='blood')
    meta3=M.metabolites.get_by_id(meta_id_c1)
    reaction.add_metabolites({meta: -1.0, meta3:1})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])

In [31]:
BloodMeta1=[meta for meta in M.metabolites if '_BD' in meta.id
            and '34dhphe' not in meta.id 
            and 'phpyr_BD'not in meta.id]
len(BloodMeta1)

2131

In [32]:
### Add some exchange reactions (all together) in the blood
for META in BloodMeta1:
    meta_id_c = META.id
    from cobra import Model, Reaction, Metabolite
    # Adding exchange reaction
    rxn_id = f'EX_{meta_id_c}'
    reaction = Reaction(rxn_id)
    reaction.name = f'Exchange of {meta_id_c}'
    reaction.lower_bound = 0  
    reaction.upper_bound = 1000
    # Create metabolite object
    meta=M.metabolites.get_by_id(META.id)
    reaction.add_metabolites({meta: -1.0})
    reaction.gene_reaction_rule = ''
    # Add reaction to model
    M.add_reactions([reaction])

In [33]:
## O2 uptake from blood and not from liver, the rest medium metabolites take up from liver
M.reactions.EX_o2_e.lower_bound=0
M.reactions.EX_o2_BD.lower_bound=-1000

In [34]:
M.solver='glpk'
sol = M.optimize()
sol.objective_value

0.16888639687627724

In [35]:
M.reactions.EX_To1_BD.upper_bound=0
M.reactions.EX_To2_BD.upper_bound=0
M.reactions.EX_To3_BD.upper_bound=0
M.reactions.EX_To4_BD.upper_bound=0
M.reactions.EX_To5_BD.upper_bound=0
M.reactions.EX_To6_BD.upper_bound=0
M.reactions.EX_To7_BD.upper_bound=0
M.reactions.EX_To8_BD.upper_bound=0
M.reactions.EX_34dhphe_BD.upper_bound=1000
M.reactions.EX_phpyr_BD.upper_bound=1000

In [36]:
SecretedRXN=[rxn for rxn in M.reactions if M.reactions.get_by_id(rxn.id).upper_bound>0 and 'EX_' in rxn.id]
len(SecretedRXN)

2125

In [37]:
M.reactions.get_by_id('34dhpheTR_TOB').lower_bound=-1000

In [38]:
cobra.io.save_json_model(M,'Recon3DplusthreeCompetitive.json')